Se você estiver rodando no Google Colab, esse trecho monta o Google Drive para acessar os arquivos diretamente de lá.

São importadas as bibliotecas principais:

- Pandas para manipulação de dados

- Numpy para operações matemáticas e arrays
- OS para manipulação de arquivos e diretórios

In [1]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import os


Mounted at /content/drive


Define os diretórios onde estão armazenados os arquivos CSV contendo os dados dos motores:

- path_healthy: diretório com os dados de motores saudáveis.

- path_faulty: diretório com os dados de motores com falhas.


In [2]:
# Definir caminhos das pastas no Drive
path_healthy = "/content/drive/MyDrive/motor_dataset/saudavel"  # Ajuste para seu diretório
path_faulty = "/content/drive/MyDrive/motor_dataset/problemas"


Essa função carrega os arquivos CSV do diretório fornecido (folder_path) e aplica algumas transformações:

- Remove espaços extras nos nomes das colunas;
- Exclui a coluna "Time Stamp", caso ela exista;
- Adiciona um rótulo (label), onde:
  - 0 representa motores saudáveis;
  - 1 representa motores com falhas.


Após carregar todos os arquivos, os dados são concatenados em um único DataFrame.

A função load_data() é usada para carregar:

- df_healthy: DataFrame contendo os dados de motores saudáveis;
- df_faulty: DataFrame contendo os dados de motores com falhas.
- df.head(): Mostra as primeiras 5 linhas do DataFrame.
- df['label'].value_counts(): Conta quantas amostras existem de cada classe (saudável e falha).

In [3]:
# Função para carregar os arquivos CSV
def load_data(folder_path, label):
    data_list = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder_path, file))

            # Remover espaços extras nos nomes das colunas
            df.columns = df.columns.str.strip()

            # Remover coluna "Time Stamp" se existir
            if "Time Stamp" in df.columns:
                df = df.drop(columns=["Time Stamp"])

            # Adicionar rótulo (0 = Saudável, 1 = Falha)
            df['label'] = label
            data_list.append(df)

    return pd.concat(data_list, ignore_index=True)

# Carregar os dados saudáveis e com falha
df_healthy = load_data(path_healthy, label=0)  # Motores saudáveis (rótulo 0)
df_faulty = load_data(path_faulty, label=1)  # Motores com falha (rótulo 1)

# Combinar os dois datasets
df = pd.concat([df_healthy, df_faulty], ignore_index=True)

# Exibir as primeiras linhas para verificar os dados
print(df.head())

# Verificar as quantidades de cada classe
print(df['label'].value_counts())


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/motor_dataset/saudavel'

In [ ]:
# Separar features (correntes A, B, C) e rótulos (label)
X = df[['Current-A', 'Current-B', 'Current-C']].values
y = df['label'].values

# Verificar formato
print(f"Formato de X: {X.shape}, Formato de y: {y.shape}")


Train_test_split() do sklearn para dividir os dados:
70% para treinamento (X_train, y_train);
30% para teste (X_test, y_test);
stratify=y garante que a proporção de classes seja mantida nos dois conjuntos

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Ajustar o formato para CNN 1D
X_train = X_train.reshape(-1, 3, 1)
X_test = X_test.reshape(-1, 3, 1)

# Exibir as dimensões finais
print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Calcular os pesos das classes
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=df['label'].values)

# Converter para dicionário para o Keras
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

# Exibir os pesos calculados
print(f"Pesos das classes: {class_weights_dict}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Input, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# Criar o modelo CNN 1D com Regularização L2 e Dropout
model = Sequential([
    Input(shape=(3, 1)),  # Entrada com 3 variáveis (Current-A, Current-B, Current-C)

    # Camada convolucional com Regularização L2
    Conv1D(filters=32, kernel_size=2, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),  # Dropout aumentado para evitar overfitting

    Flatten(),

    # Camada densa com Regularização L2
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),

    Dense(1, activation='sigmoid')  # Saída binária (0 = Saudável, 1 = Falha)
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Adicionar Early Stopping para evitar overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Treinar o modelo com os pesos das classes e Early Stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Pode rodar até 50 épocas, mas para antes se necessário
    validation_data=(X_test, y_test),
    batch_size=32,
    class_weight=class_weights_dict,  # Usando pesos para balancear os dados
    callbacks=[early_stop]  # Adicionando Early Stopping
)


In [ ]:
# Treinar o modelo com os pesos das classes
history = model.fit(
    X_train, y_train,
    epochs=15,
    validation_data=(X_test, y_test),
    batch_size=32,
    class_weight=class_weights_dict  # Passando os pesos calculados
)


In [ ]:
# Avaliação no conjunto de teste
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {accuracy:.2f}")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Fazer previsões no conjunto de teste
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Criar a matriz de confusão
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.title("Matriz de Confusão - Modelo com Class Weights")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plotar a perda (loss) do treino e validação
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.title("Gráfico de Perda (Overfitting se a validação for muito maior)")
plt.show()

# Plotar a acurácia do treino e validação
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.title("Gráfico de Acurácia (Overfitting se treino for muito maior)")
plt.show()



In [ ]:
import pandas as pd
import numpy as np
import os

# Caminho do novo conjunto de falhas
path_new_faulty = "/content/drive/MyDrive/motor_dataset/validacao"

# Função para carregar os dados das novas falhas
def load_faulty_data(folder_path, label=1):
    data_list = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder_path, file))

            # Remover espaços nos nomes das colunas
            df.columns = df.columns.str.strip()

            # Remover "Time Stamp", se existir
            if "Time Stamp" in df.columns:
                df = df.drop(columns=["Time Stamp"])

            df['label'] = label  # Todos são falhas (1)
            data_list.append(df)

    return pd.concat(data_list, ignore_index=True)

# Carregar os novos dados de falha
df_new_faulty = load_faulty_data(path_new_faulty, label=1)

# Separar as features
X_new_faulty = df_new_faulty[['Current-A', 'Current-B', 'Current-C']].values
y_new_faulty = df_new_faulty['label'].values  # Todos são falhas (1)

# Ajustar o formato para CNN 1D
X_new_faulty = X_new_faulty.reshape(-1, 3, 1)

# Verificar formato
print(f"Formato dos novos dados de falha: {X_new_faulty.shape}")


In [ ]:
# Avaliação do modelo nos novos dados de falha
loss_new_faulty, accuracy_new_faulty = model.evaluate(X_new_faulty, y_new_faulty)
print(f"Acurácia nos novos dados de falha: {accuracy_new_faulty:.2f}")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Fazer previsões nos novos dados de falha
y_pred_new_faulty = (model.predict(X_new_faulty) > 0.5).astype("int32")

# Criar a matriz de confusão
cm = confusion_matrix(y_new_faulty, y_pred_new_faulty)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.title("Matriz de Confusão - Teste com Novos Dados de Falha")
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import os

# Caminho dos dados saudáveis e de falha
path_healthy = "/content/drive/MyDrive/motor_dataset/saudavel"
path_faulty = "/content/drive/MyDrive/motor_dataset/validacao"

# Função para carregar os dados
def load_data(folder_path, label):
    data_list = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder_path, file))

            # Remover espaços nos nomes das colunas
            df.columns = df.columns.str.strip()

            # Remover "Time Stamp", se existir
            if "Time Stamp" in df.columns:
                df = df.drop(columns=["Time Stamp"])

            df['label'] = label  # 0 = Saudável, 1 = Falha
            data_list.append(df)

    return pd.concat(data_list, ignore_index=True)

# Carregar dados saudáveis e de falha
df_healthy = load_data(path_healthy, label=0)  # Motores saudáveis (0)
df_faulty = load_data(path_faulty, label=1)  # Motores com falha (1)

# Combinar os dois conjuntos para criar dados mistos
df_mixed = pd.concat([df_healthy, df_faulty], ignore_index=True)

# Embaralhar os dados para evitar padrões na ordem
df_mixed = df_mixed.sample(frac=1, random_state=42).reset_index(drop=True)

# Separar as features e os rótulos
X_mixed = df_mixed[['Current-A', 'Current-B', 'Current-C']].values
y_mixed = df_mixed['label'].values  # 0 = Saudável, 1 = Falha

# Ajustar formato para CNN 1D
X_mixed = X_mixed.reshape(-1, 3, 1)

# Verificar formato final dos dados mistos
print(f"Formato dos dados mistos: {X_mixed.shape}")
print(f"Distribuição das classes: {np.unique(y_mixed, return_counts=True)}")


In [ ]:
# Avaliação do modelo nos dados mistos
loss_mixed, accuracy_mixed = model.evaluate(X_mixed, y_mixed)
print(f"Acurácia nos dados mistos: {accuracy_mixed:.2f}")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Fazer previsões nos dados mistos
y_pred_mixed = (model.predict(X_mixed) > 0.5).astype("int32")

# Criar a matriz de confusão
cm = confusion_matrix(y_mixed, y_pred_mixed)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.title("Matriz de Confusão - Teste com Dados Mistos (Saudáveis + Falhas)")
plt.show()
